In [1]:
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Nombre *Rutas*

In [3]:
ruta_hoteles= '/content/drive/MyDrive/Datasets_PF/hotelesv2.parquet'
ruta_reviews= '/content/drive/MyDrive/Datasets_PF/all-reviews.parquet'

## Carga de Data Frame

In [4]:
df1= pd.read_parquet(ruta_hoteles)
df2= pd.read_parquet(ruta_reviews)

## Columnas de Hoteles

In [5]:
df1.columns

Index(['name', 'gmap_id', 'descrption', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'facilities', 'url', 'County', 'City',
       'State', 'Country'],
      dtype='object')

## Elimino Columnas Innecesarias

In [6]:
columnas_a_eliminar = ['latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'url', 'County', 'City', 'State', 'Country']
df1= df1.drop(columns=columnas_a_eliminar)
df1.columns

Index(['name', 'gmap_id', 'descrption', 'category', 'facilities'], dtype='object')

## Columnas de Reviews

In [7]:
df2.columns

Index(['review_id', 'trash', 'user_id', 'name', 'time', 'rating', 'text',
       'gmap_id', 'facilities', 'url'],
      dtype='object')

## Elimino Columnas Innecesarias

In [8]:
columnas_a_eliminar2 = ['review_id', 'trash', 'user_id', 'name', 'time', 'rating', 'facilities', 'url']
df2= df2.drop(columns=columnas_a_eliminar2)
df2.columns

Index(['text', 'gmap_id'], dtype='object')

## Numero de filas en fd1 y df2

In [9]:
num_filas_df1 = df1.shape[0]
num_filas_df2 = df2.shape[0]

print("Número de filas en df1:", num_filas_df1)
print("Número de filas en df2:", num_filas_df2)


Número de filas en df1: 7637
Número de filas en df2: 2095


## Valores Unicos segun gmap_id

In [10]:
# Cantidad de valores únicos de 'gmap_id' en df1 y df2
cantidad_unicos_df1 = df1['gmap_id'].nunique()
cantidad_unicos_df2 = df2['gmap_id'].nunique()

print("Cantidad de 'gmap_id' únicos en df1:", cantidad_unicos_df1)
print("Cantidad de 'gmap_id' únicos en df2:", cantidad_unicos_df2)



Cantidad de 'gmap_id' únicos en df1: 7637
Cantidad de 'gmap_id' únicos en df2: 2095


## Coincidencia en gmap_ip de df1 y df2

In [11]:
# Encontrar los 'gmap_id' que coinciden en df1 y df2
coincidencias = df1['gmap_id'].isin(df2['gmap_id'])

# Contar el número de coincidencias
cantidad_coincidencias = coincidencias.sum()

print("Cantidad de 'gmap_id' coincidentes en df1 y df2:", cantidad_coincidencias)


Cantidad de 'gmap_id' coincidentes en df1 y df2: 2095


## Unificacion

In [12]:
# Realizar un left join entre df1 y df2 en la columna 'gmap_id'
df3 = df1.merge(df2, on='gmap_id', how='left')

# Reemplazar los valores NaN con valores vacíos en las columnas de df2
df3.fillna('', inplace=True)

df3.columns

Index(['name', 'gmap_id', 'descrption', 'category', 'facilities', 'text'], dtype='object')

## Nuemero de filas en la unificacion

In [13]:
num_filas_df = df3.shape[0]

print("Número de filas en df3:", num_filas_df)

Número de filas en df3: 7637


## Funcion para buscar palabras claves

In [27]:
def buscar_palabras_clave(df, palabras_a_buscar=None, columnas=None):

    # Solicitar al usuario las palabras clave si no se proporcionaron
    if not palabras_a_buscar:
        palabras_a_buscar = input("Ingresa las palabras clave separadas por coma: ").split(',')

    # Crear una nueva columna llamada 'amenidades' y guardar las palabras clave como lista
    df['amenidades'] = df[columnas].apply(lambda row: [word for word in palabras_a_buscar if any(word.lower() in str(cell).lower() for cell in row)], axis=1)

    # Reemplazar las listas vacías por celdas vacías
    df['amenidades'] = df['amenidades'].apply(lambda x: '' if len(x) == 0 else ', '.join(x))

    return df


## Palabras Clavez y Columnas a examinar

In [28]:
palabras_a_buscar = ['wifi', 'gym', 'restaurant', 'piscina', 'laundry', 'kindergarten', 'pool', 'parking lot',
                     'massage', 'casino', 'resort', 'fitness', 'tennis', 'taxi', 'Swimming', 'food', 'parking', 'central heating', 'central heat', 'spa']
columnas = ['descrption', 'category', 'facilities', 'text']

## Puesta en marcha de la funcion

In [29]:
df = buscar_palabras_clave(df3, palabras_a_buscar, columnas)

In [31]:
df.columns

Index(['name', 'gmap_id', 'descrption', 'category', 'facilities', 'text',
       'amenidades'],
      dtype='object')

In [30]:
num_filas_df = df.shape[0]

print("Número de filas en df:", num_filas_df)

Número de filas en df: 7637


In [33]:
# Después de haber llamado a la función buscar_palabras_clave
celdas_vacias = df['amenidades'].value_counts().get('', 0)

print("Número de celdas vacías en la columna 'amenidades':", celdas_vacias)


Número de celdas vacías en la columna 'amenidades': 4589


In [43]:
final_guardar = '/content/drive/MyDrive/Datasets_PF/amenidades.xlsx'
df.to_excel(final_guardar, index=False)